<a href="https://colab.research.google.com/github/Julita257/UMwF/blob/Case-2/Case_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Przegląd danych:

*   Podaj liczbę filmów Sci-Fi
*   Pokaż rozkład ocen komedii
*   Podaj średnią ocen wszystkich filmów akcji oraz 3 filmy najwyżej ocenianych????? ***(Dopytać się)***

2. System rekomendacyjny

*  Zbuduj system na podstawie algorytmu SVD oraz kNNwithMeans
*  Czym różni się algorytm kNN with means od standardowego kNN?
*  Wykorzystaj metodę hiperparametryzacji GridSearch do wyboru liczby sąsiadów od 2-6
*  W ocenie metod wykorzystaj walidację krzyżową
3. Podaj rekomendacje po obejrzeniu filmu: Jumanji  oraz Flint

1. Dane

In [1]:
import pandas as pd


In [2]:
ratings = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv')
movies = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv')

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
all_scores = ratings['rating'].count()
all_scores

100836

In [6]:
movie_count = movies['movieId'].count()
movie_count

9742

In [7]:
user_count = len(ratings['userId'].unique())
user_count

610

In [8]:
AVG_score_user = all_scores / user_count
AVG_score_user

165.30491803278687

In [9]:
AVG_score_movie = all_scores / movie_count
AVG_score_movie

10.350646684459043

In [10]:
combined_data = pd.merge(ratings, movies, on='movieId')
combined_data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


1. Przegląd danych

In [11]:
sci_fi_movies = combined_data[combined_data['genres'].str.contains('Sci-Fi')]
sci_fi_count = sci_fi_movies['movieId'].nunique()

In [12]:
print(f'Liczba filmów Sci-Fi: {sci_fi_count}')

Liczba filmów Sci-Fi: 980


In [13]:
comedy_movies = combined_data[combined_data['genres'].str.contains('Comedy')]
comedy_distribution = comedy_movies['rating'].value_counts(normalize=True).sort_index()

In [14]:
comedy_distribution

rating
0.5    0.016183
1.0    0.033723
1.5    0.022918
2.0    0.087189
2.5    0.064784
3.0    0.212685
3.5    0.130233
4.0    0.247331
4.5    0.071544
5.0    0.113410
Name: proportion, dtype: float64

In [15]:
action_movies = combined_data[combined_data['genres'].str.contains('Action')]
AVG_score_action = action_movies['rating'].mean()
AVG_score_action

3.447984331646809

Zwykła średnia

In [16]:
average_ratings = action_movies.groupby(['movieId', 'title'])['rating'].mean().reset_index()
sorted_movies = average_ratings.sort_values(by='rating', ascending=False)
top_three_action_movies = sorted_movies.head(3)
top_three_action_movies

,movieId,title,rating
1239,72142,Love Exposure (Ai No Mukidashi) (2008),5.0
1692,138632,Tokyo Tribe (2014),5.0
1593,115727,Crippled Avengers (Can que) (Return of the 5 D...,5.0


średnia bayesowska

In [17]:
C = action_movies['rating'].mean()

m = action_movies['movieId'].value_counts().median()
rating_stats = action_movies.groupby('movieId').agg(
    rating_count=('rating', 'count'),
    rating_avg=('rating', 'mean')
).reset_index()

rating_stats['bayesian_avg'] = (rating_stats['rating_count'] * rating_stats['rating_avg'] + m * C) / (rating_stats['rating_count'] + m)

average_ratings = pd.merge(rating_stats, action_movies[['movieId', 'title']].drop_duplicates(), on='movieId')

sorted_movies = average_ratings.sort_values(by='bayesian_avg', ascending=False)

top_three_action_movies = sorted_movies.head(3)

In [18]:
print(top_three_action_movies[['title', 'bayesian_avg']])

                                          title  bayesian_avg
362                           Fight Club (1999)      4.254439
36    Star Wars: Episode IV - A New Hope (1977)      4.215781
1130                    Dark Knight, The (2008)      4.212597


2. System rekomendacyjny

In [19]:
! pip install scikit-surprise
from surprise import Dataset, Reader
from surprise import SVD, KNNWithMeans
from surprise.model_selection import cross_validate

In [20]:
reader = Reader(rating_scale=(combined_data['rating'].min(), combined_data['rating'].max()))
data = Dataset.load_from_df(combined_data[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()

In [21]:
svd = SVD()
svd.fit(trainset)
SVD_result = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8741  0.8751  0.8722  0.8739  0.8746  0.8740  0.0010  
MAE (testset)     0.6730  0.6734  0.6691  0.6713  0.6727  0.6719  0.0016  
Fit time          2.88    3.56    3.76    1.78    1.74    2.74    0.85    
Test time         0.54    0.28    0.12    0.12    0.14    0.24    0.16    


In [22]:
knn_means = KNNWithMeans()
knn_means.fit(trainset)
knn_result = cross_validate(knn_means, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8976  0.8984  0.9076  0.8840  0.9013  0.8978  0.0077  
MAE (testset)     0.6868  0.6858  0.6918  0.6787  0.6886  0.6863  0.0043  
Fit time          0.15    0.28    0.30    0.36    0.19    0.25    0.08    
Test time         2.27    4.38    3.34    2.51    1.48    2.80    0.99    


In [23]:
from surprise.model_selection import GridSearchCV

In [24]:
param_grid = {'k': [2, 3, 4, 5, 6], 'sim_options': {'name': ['pearson_baseline'], 'user_based': [False]}}
gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse', 'mae'], cv=10)
gs.fit(data)
print(f"Najlepsze parametry: {gs.best_params['rmse']}")
#Parametry ograniczone ze względu na długość obliczeń

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

In [39]:
#param_grid = {
  #  'n_factors': [50, 100, 150],
  #  'n_epochs': [20, 30],
  #  'lr_all': [0.005, 0.008],
 #   'reg_all': [0.02, 0.06]
#}
param_grid = { # zmniejszona ilość parametrów, ze względu na długość obliczeń
    'n_factors': [150],
    'n_epochs': [30],
    'lr_all': [0.008],
    'reg_all': [0.06]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=10)

gs.fit(data)

print("Najlepsze RMSE:", gs.best_score['rmse'])
print("Najlepsze parametry dla RMSE:", gs.best_params['rmse'])

# Użyj najlepszego zestawu parametrów do dalszych predykcji
best_svd = gs.best_estimator['rmse']

Najlepsze RMSE: 0.8477579438312632
Najlepsze parametry dla RMSE: {'n_factors': 150, 'n_epochs': 30, 'lr_all': 0.008, 'reg_all': 0.06}


In [26]:
best_knn_params = {
    'k': 6,
    'sim_options': {
        'name': 'pearson_baseline',
        'user_based': False
    }
}

knn_with_means = KNNWithMeans(k=best_knn_params['k'], sim_options=best_knn_params['sim_options'])
knn_with_means.fit(trainset)

In [41]:
best_svd = {'n_factors': 150, 'n_epochs': 30, 'lr_all': 0.008, 'reg_all': 0.06}
svd = SVD(n_factors=best_svd['n_factors'], n_epochs=best_svd['n_epochs'],
          lr_all=best_svd['lr_all'], reg_all=best_svd['reg_all'])

svd.fit(trainset)

Walidacja krzyżowa

In [43]:
svd_results = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)
knn_results = cross_validate(knn_with_means, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.8449  0.8460  0.8541  0.8407  0.8404  0.8576  0.8406  0.8539  0.8535  0.8631  0.8495  0.0076  
MAE (testset)     0.6462  0.6476  0.6534  0.6436  0.6405  0.6564  0.6478  0.6560  0.6519  0.6584  0.6502  0.0057  
Fit time          3.42    4.56    3.75    3.42    4.07    3.71    3.44    3.44    3.90    3.42    3.71    0.36    
Test time         0.07    0.11    0.06    0.06    0.50    0.11    0.06    0.10    0.10    0.06    0.12    0.13    
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases u

In [44]:
print("Średni RMSE dla SVD: ", svd_results['test_rmse'].mean())
print("Średni MAE dla SVD: ", svd_results['test_mae'].mean())

print("Średni RMSE dla KNN with Means: ", knn_results['test_rmse'].mean())
print("Średni MAE dla KNN with Means: ", knn_results['test_mae'].mean())


Średni RMSE dla SVD:  0.849479674736318
Średni MAE dla SVD:  0.6501747640195602
Średni RMSE dla KNN with Means:  0.9020255138215625
Średni MAE dla KNN with Means:  0.6844794713513871


In [45]:
import numpy as np
def generate_recommendations(movie_title, data, svd_model, top_n=3):
    movie_ids = data[data['title'].str.contains(movie_title, case=False, na=False)]['movieId'].unique()

    users_who_liked_movie = data[(data['movieId'].isin(movie_ids)) & (data['rating'] > data['rating'].mean())]['userId'].unique()

    recommendations = []
    for movie_id in data['movieId'].unique():
        if movie_id not in movie_ids:
            mean_rating = np.mean([svd_model.predict(user_id, movie_id).est for user_id in users_who_liked_movie])
            recommendations.append((movie_id, mean_rating))

    recommendations.sort(key=lambda x: x[1], reverse=True)
    top_recommendations = recommendations[:top_n]

    top_recommendations = [(data[data['movieId'] == movie_id]['title'].iloc[0], rating) for movie_id, rating in top_recommendations]

    return top_recommendations

# Wywołanie funkcji dla "Jumanji" i "Flint"
jumanji_recommendations = generate_recommendations("Jumanji", combined_data, svd)
flint_recommendations = generate_recommendations("Flint", combined_data, svd)

print("Rekomendacje po obejrzeniu 'Jumanji':")
for title, rating in jumanji_recommendations:
    print(f"{title}: {rating:.2f}")

print("\nRekomendacje po obejrzeniu 'Flint':")
for title, rating in flint_recommendations:
    print(f"{title}: {rating:.2f}")


Rekomendacje po obejrzeniu 'Jumanji':
Guess Who's Coming to Dinner (1967): 4.51
Shawshank Redemption, The (1994): 4.50
His Girl Friday (1940): 4.50

Rekomendacje po obejrzeniu 'Flint':
Day of the Doctor, The (2013): 4.60
Charlie Brown Christmas, A (1965): 4.56
Grand Day Out with Wallace and Gromit, A (1989): 4.55


In [50]:
def generate_recommendations(movie_title, data, knn_model, top_n=3):
    movie_ids = data[data['title'].str.contains(movie_title, case=False, na=False)]['movieId'].unique()
    users_who_liked_movie = data[(data['movieId'].isin(movie_ids)) & (data['rating'] > data['rating'].mean())]['userId'].unique()

    recommendations = []
    for movie_id in data['movieId'].unique():
        if movie_id not in movie_ids:
            mean_rating = np.mean([knn_model.predict(user_id, movie_id).est for user_id in users_who_liked_movie])
            recommendations.append((movie_id, mean_rating))

    recommendations.sort(key=lambda x: x[1], reverse=True)
    top_recommendations = recommendations[:top_n]
    top_recommendations = [(data[data['movieId'] == movie_id]['title'].iloc[0], rating) for movie_id, rating in top_recommendations]

    return top_recommendations


jumanji_recommendations = generate_recommendations("Jumanji", combined_data, knn_with_means)
flint_recommendations = generate_recommendations("Flint", combined_data, knn_with_means)

print("Rekomendacje po obejrzeniu 'Jumanji':")
for title, rating in jumanji_recommendations:
    print(f"{title}: {rating:.2f}")

print("\nRekomendacje po obejrzeniu 'Flint':")
for title, rating in flint_recommendations:
    print(f"{title}: {rating:.2f}")

Rekomendacje po obejrzeniu 'Jumanji':
Day of the Doctor, The (2013): 4.77
Belle époque (1992): 4.65
Last Detail, The (1973): 4.64

Rekomendacje po obejrzeniu 'Flint':
Strictly Sexual (2008): 4.91
Mr. Woodcock (2007): 4.89
Peaceful Warrior (2006): 4.86


Różnice między KNN i KNN with mean:
1. KNN with mean posiada dodatkową korektę mającą na celu uwzględnienie różnych skłonności oceniania przez użytkowników.
2. KNN with Means w przeciwieństwie do KNN dostosowuje oceny na podstawie indywidualnych średnich użytkowników przed wykorzystaniem ich do prognozowania.
3. KNN with Means zwykle daje lepsze wyniki w środowiskach, gdzie różni użytkownicy mają różne skłonności oceniania, ponieważ normalizacja pomaga zredukować te różnice.
